In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os


In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

x_train_cnn = np.expand_dims(x_train, -1)
x_test_cnn  = np.expand_dims(x_test, -1)


In [ ]:
mlp_model = keras.Sequential([
    layers.Flatten(input_shape=(28,28)),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

mlp_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

mlp_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


In [ ]:
cnn_model = keras.Sequential([
    layers.Conv2D(16, 3, activation="relu", input_shape=(28,28,1)),
    layers.MaxPooling2D(2),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

cnn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

cnn_model.fit(x_train_cnn, y_train, epochs=5, batch_size=64, validation_split=0.1)


In [ ]:
mlp_model.save("mlp_model.h5")
cnn_model.save("cnn_model.h5")


In [ ]:
def rep_data():
    for i in range(100):
        yield [x_train[i:i+1]]

rep_ds = tf.data.Dataset.from_generator(
    rep_data,
    output_types=tf.float32,
    output_shapes=[(1,28,28)]
)


In [ ]:
converter_mlp = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
converter_mlp.optimizations = [tf.lite.Optimize.DEFAULT]
converter_mlp.representative_dataset = rep_ds
converter_mlp.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_mlp.inference_input_type = tf.int8
converter_mlp.inference_output_type = tf.int8

tflite_mlp = converter_mlp.convert()

with open("mlp_model.tflite", "wb") as f:
    f.write(tflite_mlp)


In [ ]:
converter_cnn = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
converter_cnn.optimizations = [tf.lite.Optimize.DEFAULT]
converter_cnn.representative_dataset = rep_ds
converter_cnn.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_cnn.inference_input_type = tf.int8
converter_cnn.inference_output_type = tf.int8

tflite_cnn = converter_cnn.convert()

with open("cnn_model.tflite", "wb") as f:
    f.write(tflite_cnn)


In [ ]:
def size_mb(path):
    return os.path.getsize(path) / (1024 * 1024)

print("MLP h5 size (MB):", size_mb("mlp_model.h5"))
print("CNN h5 size (MB):", size_mb("cnn_model.h5"))
print("MLP tflite size (MB):", size_mb("mlp_model.tflite"))
print("CNN tflite size (MB):", size_mb("cnn_model.tflite"))
